In [1]:
!pip install nltk

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
from nltk import ngrams
from sklearn.utils import shuffle
import numpy as np
import tensorflow as tf

In [ ]:
with open('/content/drive/My Drive/train-bel-clear.txt', mode='r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
text = '\n'.join(shuffle(text.split('\n'), random_state=20))

In [8]:
%%time 
y = [1 if next_char==' ' else 0 for cur_char, next_char in zip(text[:-1], text[1:]) if cur_char != ' ']

CPU times: user 21.3 s, sys: 1.81 s, total: 23.1 s
Wall time: 23.1 s


In [9]:
border = int(len(x) * 0.7)
border += x[border:].find('\n')
x[border]

'\n'

In [ ]:
text_train, text_test = x[:border], x[border+1:]
y_train, y_test = y[:border-1], y[border+1:]

In [ ]:
split_num = 0.01
split_len = 100
borders = [int(len(text_train) * split_num * i) for i in range(1, split_len)]
borders = [0] + [border + text_train[border:].find('\n') 
                                    for border in borders] + [len(text_train)+1]
text_list = [text_train[border_l:border_r]
             for border_l, border_r in zip(borders[:-1], borders[1:])]
y_list = [y_train[border_l:border_r]
             for border_l, border_r in zip(borders[:-1], borders[1:])]

In [12]:
borders[:10]

[0,
 978289,
 1956171,
 2934373,
 3912474,
 4890484,
 5868493,
 6846789,
 7824646,
 8802715]

In [13]:
vocab = sorted(set(text_train))
print ('{} unique characters'.format(len(vocab)))

348 unique characters


In [14]:


%%time
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_list = [[char2idx.get(c) for c in part] for part in text_list] 
text_test = [char2idx.get(c, len(vocab)) for c in text_test]

CPU times: user 27.6 s, sys: 492 ms, total: 28 s
Wall time: 28.1 s


In [15]:
text_list[0][:10]

[233, 263, 274, 255, 272, 252, 274, 260, 265, 266]

In [ ]:
BATCH_SIZE = 16

def get_preproc(df, batch=BATCH_SIZE):
    df = df.batch(BATCH_SIZE, drop_remainder=True)
    return df

In [ ]:
# LENGTH = 15
# shape = ((LENGTH, 1), (LENGTH-1, 1))
# gen_dataset = ((np.array(x_i).reshape(LENGTH, 1), np.array(y_i).reshape(LENGTH-1, 1) 
#                 for x_i, y_i in zip(ngrams(text_train, LENGTH), ngrams(y_train, LENGTH-1))) 
# df = tf.data.Dataset.from_generator(generator=lambda: gen_dataset,
#                                    output_types=(tf.int16, tf.int8),
#                                     output_shapes=shape)

In [32]:
%%time
LENGTH = 15
gen_list = [((list(x_i), y_i) for x_i, y_i in 
                        zip(ngrams(part_x, LENGTH), part_y[LENGTH-1:])) 
            for part_x, part_y in zip(text_list, y_list)
]
# df_list = [tf.data.Dataset.from_generator(generator=lambda: gen_dataset,
#                                           output_types=(tf.int16, tf.int8),
#                                           output_shapes=((LENGTH,), ()))
#             for gen_dataset in gen_list
# ]
ngrams_1 = list(map(list, list(ngrams(text_list[0], LENGTH))))
df_1 = tf.data.Dataset.from_tensor_slices((ngrams_1, y_list[0][LENGTH-1:]))
df_1

CPU times: user 1min 2s, sys: 572 ms, total: 1min 2s
Wall time: 1min 2s


In [33]:
df_1

<TensorSliceDataset shapes: ((15,), ()), types: (tf.int32, tf.int32)>

In [ ]:
# df_list = list(map(get_preproc, df_list))
# df_list[0]

In [ ]:
# list(df_list[0].take(1).as_numpy_iterator())

In [36]:
df_1 = get_preproc(df_1)
df_1

<BatchDataset shapes: ((16, 15), (16,)), types: (tf.int32, tf.int32)>

In [48]:
list(df_1.take(1).as_numpy_iterator())

[(array([[233, 263, 274, 255, 272, 252, 274, 260, 265, 266, 226, 242, 219,
          220, 248],
         [263, 274, 255, 272, 252, 274, 260, 265, 266, 226, 242, 219, 220,
          248, 263],
         [274, 255, 272, 252, 274, 260, 265, 266, 226, 242, 219, 220, 248,
          263, 248],
         [255, 272, 252, 274, 260, 265, 266, 226, 242, 219, 220, 248, 263,
          248, 257],
         [272, 252, 274, 260, 265, 266, 226, 242, 219, 220, 248, 263, 248,
          257, 266],
         [252, 274, 260, 265, 266, 226, 242, 219, 220, 248, 263, 248, 257,
          266, 219],
         [274, 260, 265, 266, 226, 242, 219, 220, 248, 263, 248, 257, 266,
          219, 249],
         [260, 265, 266, 226, 242, 219, 220, 248, 263, 248, 257, 266, 219,
          249, 248],
         [265, 266, 226, 242, 219, 220, 248, 263, 248, 257, 266, 219, 249,
          248, 259],
         [266, 226, 242, 219, 220, 248, 263, 248, 257, 266, 219, 249, 248,
          259, 266],
         [226, 242, 219, 220, 248, 263, 

In [ ]:
def buid_model(batch):
    vocab_size = len(vocab) + 1
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=256,
                                  batch_input_shape=[batch, None]),
    tf.keras.layers.GRU(units=512,
                        return_sequences=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.GRU(units=128,
                        return_sequences=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(rate=0.05),
    tf.keras.layers.Dense(1, activation='relu')])
    return model

In [50]:
model = buid_model(BATCH_SIZE)
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (16, None, 256)           89344     
_________________________________________________________________
gru_2 (GRU)                  (16, None, 512)           1182720   
_________________________________________________________________
dense_3 (Dense)              (16, None, 512)           262656    
_________________________________________________________________
gru_3 (GRU)                  (16, None, 128)           246528    
_________________________________________________________________
dropout_2 (Dropout)          (16, None, 128)           0         
_________________________________________________________________
dense_4 (Dense)              (16, None, 1)             129       
Total params: 1,781,377
Trainable params: 1,781,377
Non-trainable params: 0
____________________________________________

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    verbose=0)

In [53]:
EPOCH = 5
history = model.fit(df_1,
                    shuffle=False, 
                    epochs=EPOCH,
                    callbacks=[checkpoint_callback])

Epoch 1/5
61142/61142 [==============================] - 448s 7ms/step - loss: 2.7081
Epoch 2/5
61142/61142 [==============================] - 444s 7ms/step - loss: 2.7081
Epoch 3/5
61142/61142 [==============================] - 445s 7ms/step - loss: 2.7081
Epoch 4/5
61142/61142 [==============================] - 471s 8ms/step - loss: 2.7081
Epoch 5/5
61142/61142 [==============================] - 476s 8ms/step - loss: 2.7081


In [54]:
y_pred = model.predict(df_1,verbose=1)

61142/61142 [==============================] - 196s 3ms/step


In [ ]:
y_pred[:10]